In [1]:
# Creating spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [2]:
from pyspark.ml.regression import LinearRegression

In [3]:
data = spark.read.csv ('Ecommerce_Customers.csv' , inferSchema=True , header = True)

In [4]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [5]:
data.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)]

In [6]:
# Importing vectirs from linear algebra
from pyspark.ml.linalg import Vectors
# Importing vector assembler in ml.feature
from pyspark.ml.feature import VectorAssembler

In [7]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [8]:
# To convert all the features into a feature vector
assembler = VectorAssembler(inputCols=['Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership'],outputCol='features')

In [9]:
# Transforming data into the assembler contains selected features
output = assembler.transform(data)
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [10]:
# Creating actual dataset
final_data = output.select('features','Yearly Amount Spent')
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [11]:
# Splitting training and test data set
train_data , test_data = final_data.randomSplit([0.7,0.3])
# Printing description with count of the training data
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                369|
|   mean|   497.686553907393|
| stddev|  78.40129127354486|
|    min|   266.086340948469|
|    max|  765.5184619388373|
+-------+-------------------+



In [12]:
# Giving col names and framing the linear regression
lr = LinearRegression(labelCol='Yearly Amount Spent')
# Creating training model
lr_model = lr.fit(train_data)

In [13]:
# Evaluating the model
test_results = lr_model.evaluate (test_data)
test_results.residuals.show()

C:\Users\gokulnath.kandasamy\Anaconda3\lib\site-packages\pyspark\sql\context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+-------------------+
|          residuals|
+-------------------+
| -3.544484006949176|
| 3.9718867268524036|
| -5.715765567606752|
| -7.153946754556046|
|-3.3445020685373947|
|-3.7155348005528595|
| -5.968088255866405|
| -5.484373405003112|
| 3.3855512308710445|
|-17.071176826019155|
|  18.30763334865037|
| 17.480604816459675|
| -4.466119087120262|
| -4.756442718422079|
|-2.9891720580642414|
|-17.215098290474828|
| -16.96635689804458|
|-13.720700755368853|
|  5.202543617293713|
|  4.918753554032378|
+-------------------+
only showing top 20 rows



In [14]:
test_results.rootMeanSquaredError

9.410567193817228

In [16]:
# Preparing data for the prediction
unlabeled_data = test_data.select('features')
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.8794843441274...|
|[31.0472221394875...|
|[31.0613251567161...|
|[31.1280900496166...|
|[31.2681042107507...|
|[31.4252268808548...|
|[31.5147378578019...|
|[31.5257524169682...|
|[31.5316044825729...|
|[31.5702008293202...|
|[31.6005122003032...|
|[31.6098395733896...|
|[31.6253601348306...|
|[31.7656188210424...|
|[31.8124825597242...|
|[31.8164283341993...|
|[31.9048571310136...|
|[31.9365486184489...|
|[31.9764800614612...|
|[32.0215955013870...|
+--------------------+
only showing top 20 rows



In [17]:
# Prediction using unlabeled data
predictions = lr_model.transform(unlabeled_data)
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.8794843441274...|493.75108399180385|
|[31.0472221394875...|  388.525512462169|
|[31.0613251567161...|493.27122362550836|
|[31.1280900496166...| 564.4066335016107|
|[31.2681042107507...| 426.8150352423613|
|[31.4252268808548...| 534.4822534553148|
|[31.5147378578019...| 495.7805762523278|
|[31.5257524169682...|  449.450000214885|
|[31.5316044825729...| 433.1300544984915|
|[31.5702008293202...|  563.016668967424|
|[31.6005122003032...|460.86521814244657|
|[31.6098395733896...| 427.0649448346485|
|[31.6253601348306...|380.80301984404446|
|[31.7656188210424...| 501.3105243540292|
|[31.8124825597242...|395.79951704186146|
|[31.8164283341993...| 518.3375897941312|
|[31.9048571310136...| 490.9162143208607|
|[31.9365486184489...|440.92008565069705|
|[31.9764800614612...| 325.3919024168065|
|[32.0215955013870...|  516.653421203795|
+--------------------+------------